In [1]:

%reload_ext autoreload
%autoreload 2

import os
import sys

# # Add the root folder to the module search path
# # Get the current directory
# current_directory = os.getcwd()

# # Move two levels up (go to the parent directory of the parent directory)
# # two_levels_up_directory = os.path.dirname(os.path.dirname(current_directory))
# one_levels_up_directory = os.path.dirname(current_directory)

# print(one_levels_up_directory)

# sys.path.append(two_levels_up_directory)
sys.path.append("/home/ubuntu/git/pykoi")


/home/ubuntu/git/pykoi/example


In [1]:
import pykoi

/opt/conda/envs/pykoi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Train RLHF using the data from database

Let's take a look of the QA data and process it for training.

In [6]:
stack_exchange_df = pd.read_csv("stack_exchange_10k_dataset.csv", index_col=0)
stack_exchange_df.head()

,ID,Question,Answer,Vote Status,Timestamp
0,1,who founded YC and what does YC do,Y Combinator is a startup accelerator that pro...,down,2023-07-24 08:28:29.461045
1,2,what are the top YC companies,I am looking for a list of the top YC companie...,down,2023-07-24 08:29:17.521467
2,3,How much does top 10 YC companies worth,"According to the latest Crunchbase data, the 1...",down,2023-07-24 08:29:58.248998
3,4,What's the meaning of life,"The meaning of life is to be happy, to be usef...",down,2023-07-24 19:26:04.713297
4,5,What is the meaning of life,The meaning of life is to find your gift. The ...,up,2023-07-24 19:26:28.676427


In [8]:
from datasets import Dataset

print("My ranking database has {} samples".format(stack_exchange_df.shape[0]))
dataset = Dataset.from_dict(stack_exchange_df)
dataset

My local database has 118 samples


Dataset({
    features: ['ID', 'Question', 'Answer'],
    num_rows: 118
})

### Train with RLHF

In [10]:
# run supervised finetuning
config = pykoi.RLHFConfig(base_model_path="elinas/llama-7b-hf-transformers-4.29", 
                          dataset_type="local_db",
                          
                          )
rlhf_step1_sft = pykoi.SupervisedFinetuning(config)
rlhf_step1_sft.train_and_save("./models/rlhf_step1_sft")


/opt/conda/envs/pykoi/lib/python3.10/site-packages/trl/trainer/utils.py:246: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictonnary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


My local database has 118 samples
Size of the train set: 106.               Size of the validation set: 12


Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.71s/it]
/opt/conda/envs/pykoi/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Using pad_token, but it is not set yet.
/opt/conda/envs/pykoi/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/envs/pykoi/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(
/opt/conda/envs/pykoi/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed i

Step,Training Loss,Validation Loss


: 